In [ ]:
from pwn import *
import HashTools
from tqdm import trange
for i in trange(64) :
    io = remote("127.0.0.1", 1337)

    io.recvuntil(b"Example: ")
    datas = io.recvline().decode()[:-1].split("|")
   
    original_data = datas[0].encode()
    sig = datas[1]
    append_data = b"&file=flag.txt"
    
    magic = HashTools.new("sha1")
    new_data, new_sig = magic.extension(
        secret_length=i, original_data=original_data,
        append_data=append_data, signature=sig
        )
   
        # print(str(new_data))
    send_data = str(new_data)[2:][:-1] + '|' + str(new_sig)
    user_data, received_hmac = send_data.rsplit("|", 1)
  
    try :
        user_data_bytes = bytes(user_data, "utf-8").decode("unicode_escape").encode("latin-1")
      
        io.sendline(send_data.encode())
        print(io.recvall())
        exit()
    except :
        pass
    io.close()

- Sử dụng công cụ https://kt.gy/tools.html#hash///0/
- Đầu tiên khi ta kết nối tới server, server sẽ trả về 

![image.png](image1.png)


- Ngó tới hàm handle_client() ta thấy giá trị được trả về là **original_data** và **original_hmac**

In [ ]:
def handle_client(client, addr):
    print(f"[*] Connection from {addr}")

    try:
        original_data = "count=10&lat=37.351&user_id=1&long=-119.827&file=random.txt"
        original_hmac = generate_hmac(original_data)

        client.sendall(f"Retrieve file using format: data|hmac\nExample: {original_data}|{original_hmac}\n".encode())

        data = client.recv(1024)

- Tiếp đó, để lấy được flag thì ta cần phải bypass được 2 điều kiện đó là **computed_signature = received_hmac** và **filename is True** tức là có sự xuất hiện của **flag.txt**

In [ ]:
if computed_signature != received_hmac:
    client.sendall(b"Invalid HMAC. Try again.\n")
else:
    try:
        params = dict(param.split("=") for param in user_data.split("&") if "=" in param)
        filename = params.get("file")
        if filename:
            with open(filename, "r") as f:
                content = f.read()
            client.sendall(f"File Contents:\n{content}\n".encode())
        else:
            client.sendall(b"Invalid request format.\n")
    except FileNotFoundError:
        client.sendall(b"File not found.\n")

 - Hiểu được phương pháp tấn công ta sẽ thêm các dữ liệu như trong ảnh dưới

![image.png](image2.png)


In [ ]:
new_data = "636f756e743d3130266c61743d33372e33353126757365725f69643d31266c6f6e673d2d3131392e3832372666696c653d72616e646f6d2e747874800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002182666696c653d666c61672e747874"
new_sig = b"29e0103e9ef9d3fc22f82a37461d2fb7ae5f4ab7"

print(bytes.fromhex(new_data)+ b"|" + new_sig)

b'count=10&lat=37.351&user_id=1&long=-119.827&file=random.txt\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x18&file=flag.txt|29e0103e9ef9d3fc22f82a37461d2fb7ae5f4ab7'


 - Sau khi gửi đoạn dữ liệu trên vào server đang kết nối ta thu được flag như ảnh

 ![image.png](image3.png)

 - **Chú ý:** Ở đây secret là 8 còn trong bài ta sẽ dùng vòng for hoặc while để brute force tìm ra độ dài secret

In [ ]:
# Script tham khao
from pwn import *
import HashTools
from tqdm import trange
for i in trange(64) :
    io = remote("127.0.0.1", 1337)

    io.recvuntil(b"Example: ")
    datas = io.recvline().decode()[:-1].split("|")
   
    original_data = datas[0].encode()
    sig = datas[1]
    append_data = b"&file=flag.txt"
    
    magic = HashTools.new("sha1")
    new_data, new_sig = magic.extension(
        secret_length=i, original_data=original_data,
        append_data=append_data, signature=sig
        )
   
    send_data = str(new_data)[2:][:-1] + '|' + str(new_sig)
    user_data, received_hmac = send_data.rsplit("|", 1)
  
    try :
        user_data_bytes = bytes(user_data, "utf-8").decode("unicode_escape").encode("latin-1")
      
        io.sendline(send_data.encode())
        print(io.recvall())
        exit()
    except :
        pass
    io.close()